In [3]:
import numpy as np

In [4]:
# For getting how long each group of customers would spend
np.random.uniform(30,150) # Generate one random number between 30 and 150 under uniform distribution

144.19198397661827

In [5]:
class Tables:
    def __init__(self, num, size):
        self.num=num # No. of the table
        self.size = size # Size of the table: for group of up to 2, 4 or 6.
        self.currentStatus = None # Is the table occupied or not.
        self.time_remaining = 0 
        
    def busy(self):
        if self.currentStatus != None:
            return True
        else:
            return False
    
    def startNext(self, newtask):
        self.currentStatus = newtask
        self.time_remaining = newtask.time_needs() # here maybe generate a random time from 10 - 60 mins using generat

In [33]:
def table_generator(number_tables_2, number_tables_4, number_tables_6):
    table_2_list = []
    table_4_list = []
    table_6_list = [] 
    for i in range(number_tables_2):
        table_2_list.append(Tables(i,2))
    
    for i in range(number_tables_4):
        table_4_list.append(Tables(i+number_tables_2,4))
    
    for i in range(number_tables_6):
        table_6_list.append(Tables(i+number_tables_4+number_tables_6,6))
    
    return (table_2_list, table_4_list, table_6_list)

In [38]:
table_2_list, table_4_list, table_6_list = table_generator(6,4,2)

In [39]:
table_2_list[0].num

0

In [40]:
table_4_list[0].num

2

In [70]:
class Groups:
    def __init__(self, time, size, vip):
        self.timestamp = time # Time when group registered (entered into the queue)
        self.size = size #randomly define size from 1 - 6
        self.vip = vip # Whether the group is a vip group
        self.table=None # Which table the group will be assigned to
    
    def get_stamp(self):
        return self.timestamp
    
    def get_size(self):
        return self.size
    
    def wait_time(self, current_time):
        return current_time - self.timestamp
    
    def get_vip(self):
        return self.vip

In [78]:
class Queue:
    def __init__(self):
        self.groups=[]
    
    def queue_size(self):
        return len(self.groups)
    
    def isEmpty(self):
        if len(self.groups)>0:
            return True
        else:
            False
    
    def add_queue(self, group):
        if group.get_vip():
            self.groups.insert(4,group) # If current group is a VIP group, move it forward by four groups
        else:
            self.groups.insert(0, group)
        
    
    def del_queue(self): #delete last=delete first come group 
        self.groups.pop()

In [26]:
def mod_pert_random(low, likely, high, confidence=4, samples=30):
    """Produce random numbers according to the 'Modified PERT' 
    distribution. 
    
    :param low: The lowest value expected as possible.
    :param likely: The 'most likely' value, statistically, the mode.
    :param high: The highest value expected as possible.
    :param confidence: This is typically called 'lambda' in literature 
                        about the Modified PERT distribution. The value
                        4 here matches the standard PERT curve. Higher
                        values indicate higher confidence in the mode.
                        
    Formulas from "Modified Pert Simulation" by Paulo Buchsbaum.
    """
    # Check minimum & maximum confidence levels to allow:
    confidence = min(8, confidence)
    confidence = max(2, confidence)

    mean = (low + confidence * likely + high)/(confidence + 2)

    a = (mean - low)/(high - low) * (confidence + 2)
    b = ((confidence + 1) * high - low - confidence * likely) / (high - low)
    
    beta = np.random.beta(a, b, samples)
    beta = beta * (high - low) + low
    return beta

In [79]:
def newGroup(Duration):
    size = np.random.randint(1,7,30)
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    counter = 0
    group_2 = Queue()
    group_4 = Queue()
    group_6 = Queue()
    for i in range(Duration):
        if i in timestamp_list:
            if size[counter] == 1 or size[counter] == 2:
                group_2.add_queue(Groups(i, 2, vip[counter]))
                counter += 1
            elif size[counter] == 3 or size[counter] == 4:
                group_4.add_queue(Groups(i, 4, vip[counter]))
                counter += 1
            else:
                group_6.add_queue(Groups(i, 6, vip[counter]))
                counter += 1
    return (group_2, group_4 , group_6)

In [80]:
group_2, group_4, group_6 = newGroup(240)

204